In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb

In [12]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import re


In [19]:
# Load your electric vehicle data (features and labels)
# Replace 'data.csv' with the path to your dataset.
data = pd.read_csv('Book5.csv')
data = data.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

In [20]:
# Define features and labels
X = data.drop('actual_range', axis=1)
y = data['actual_range']
print(y)

0     34.836
1     36.075
2     30.510
3     34.686
4     51.948
5     42.411
6     31.440
7     31.776
8     34.155
9     35.210
10    33.217
11    31.404
12    35.680
13    34.428
14    30.336
15    31.212
16    28.631
17    35.700
18    38.401
19    30.222
Name: actual_range, dtype: float64


In [21]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [22]:
# Train XGBoost model
xgb_model = xgb.XGBRegressor()
xgb_model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [23]:
# Train LightGBM model
lgb_model = lgb.LGBMRegressor()
lgb_model.fit(X_train, y_train)

[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 16, number of used features: 0
[LightGBM] [Info] Start training from score 34.653438
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the spl

LGBMRegressor()

In [28]:
# Combine model predictions (you can use more advanced blending strategies)
xgb_predictions = xgb_model.predict(X_test)
print(xgb_predictions)

[30.284666 32.764435 30.370464 31.333715]


In [29]:
lgb_predictions = lgb_model.predict(X_test)
print(lgb_predictions)

[34.65343761 34.65343761 34.65343761 34.65343761]


In [30]:
combined_predictions = (xgb_predictions + lgb_predictions) / 2
print(combined_predictions)

[32.46905184 33.70893621 32.51195097 32.99357653]


In [31]:
# Implement the anchor-based strategy
def anchor_adjustment(predictions, actual_range, anchor_threshold):
    adjustments = np.where(predictions > anchor_threshold, actual_range - predictions, 0)
    return predictions + adjustments

anchor_threshold = 10  # Adjust this threshold as needed
adjusted_predictions = anchor_adjustment(combined_predictions, y_test, anchor_threshold)
print(adjusted_predictions)

[34.836 35.7   31.212 36.075]


In [27]:
# Evaluate the model's performance
rmse = np.sqrt(mean_squared_error(y_test, adjusted_predictions))
print("Root Mean Squared Error:", rmse)

Root Mean Squared Error: 0.0
